---
title: "Ribasim Delwaq coupling"
---

In order to generate the Delwaq input files, we need a completed Ribasim simulation (typically one with a results folder) that ideally also includes some substances and initial concentrations. Let's take the basic test model for example, which already has set some initial concentrations.

All testmodels can be [downloaded from here](/install.qmd).

In [ ]:
from pathlib import Path

toml_path = Path("../../generated_testmodels/basic/ribasim.toml")

assert toml_path.is_file()

This Ribasim model already has substance concentrations for `Cl` and `Tracer` in the input tables, and we will use these to generate the Delwaq input files.

In [ ]:
from ribasim import Model

model = Model.read(toml_path)

display(model.basin.concentration_state)  # basin initial state
display(model.basin.concentration)  # basin boundaries
display(model.flow_boundary.concentration)  # flow boundaries
display(model.level_boundary.concentration)  # level boundaries
model.plot();  # for later comparison

In [ ]:
model.basin.profile

Let's add another tracer to the model, to setup a fraction calculation.

In [ ]:
from ribasim.delwaq import add_tracer

add_tracer(model, 11, "Foo")
add_tracer(model, 15, "Bar")
display(model.flow_boundary.concentration)  # flow boundaries
display(model.level_boundary.concentration)  # flow boundaries

model.write(toml_path)

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{toml_path.as_posix()}")',
    ],
    check=True,
)

Given the path to a completed Ribasim simulation, we can call `ribasim.delwaq.generate` for generating the required input files for Delwaq from scratch.

In [ ]:
from ribasim.delwaq import generate

output_path = Path("../../generated_testmodels/basic/delwaq")

graph, substances = generate(toml_path, output_path)

This call produces a handful of files in the user defined folder. Let's take a look at them:


In [ ]:
list(output_path.iterdir())

These files form a complete Delwaq simulation, and can be run by either pointing DIMR to the `dimr_config.xml` file or pointing Delwaq to the `delwaq.inp` file.

Note that the call to `generate` produces two output variables; `graph` and `substances` that are required for parsing the results of the Delwaq model later on. Nonetheless, we can also inspect them here, and inspect the created Delwaq network.

In [ ]:
substances  # list of substances, as will be present in the Delwaq netcdf output

As you can see, the complete substances list is a combination of user input (`Cl` and `Tracer` in the input tables), a `Continuity` tracer, and tracers for all nodetypes in the Ribasim model. The latter tracers allow for deeper inspection of the Ribasim model, such as debugging the mass balance by plotting fraction graphs. Let's inspect the `graph` next, which is the Delwaq network that was created from the Ribasim model:

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Let's draw the graph
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
nx.draw(
    graph,
    pos={k: v["pos"] for k, v in graph.nodes(data=True)},
    with_labels=True,
    labels={k: k for k, v in graph.nodes(data=True)},
    ax=ax[0],
)
ax[0].set_title("Delwaq node IDs")
nx.draw(
    graph,
    pos={k: v["pos"] for k, v in graph.nodes(data=True)},
    with_labels=True,
    labels={k: v["id"] for k, v in graph.nodes(data=True)},
    ax=ax[1],
)
ax[1].set_title("Ribasim node IDs")
fig.suptitle("Delwaq network");

Here we plotted the Delwaq network twice, with the node IDs as used by Delwaq on the left hand side, and the corresponding Ribasim node IDs on the right hand side.
As you can see, the Delwaq network is very similar to the Ribasim network, with some notable changes:

- All non-Basin or non-boundary types are removed (e.g. no more Pumps or TabulatedRatingCurves)
- Basin boundaries are split into separate nodes and links (drainage, precipitation, and evaporation, as indicated by the duplicated Basin IDs on the right hand side)
- All node IDs have been renumbered, with boundaries being negative, and Basins being positive.

## Parsing the results
With Delwaq having run, we can now parse the results using `ribasim.delwaq.parse`. This function requires the `graph` and `substances` variables that were output by `ribasim.delwaq.generate`, as well as the path to the results folder of the Delwaq simulation.

In [ ]:
from ribasim.delwaq import parse

nmodel = parse(toml_path, graph, substances, output_folder=output_path)

The parsed model is identical to the Ribasim model, with the exception of the added concentration_external table that contains all tracer results from Delwaq.

In [ ]:
display(nmodel.basin.concentration_external)
print(substances)
t = nmodel.basin.concentration_external.df
t[t.time == t.time.unique()[2]]

We can use this table to plot the results of the Delwaq model, both spatially as over time.

In [ ]:
from ribasim.delwaq import plot_fraction

plot_fraction(nmodel, 1)  # default tracers, should add up to 1
plot_fraction(nmodel, 9, ["Foo", "Bar"])  # custom tracers
plot_fraction(nmodel, 9, ["Continuity"])  # mass balance check

In [ ]:
from ribasim.delwaq import plot_spatial

plot_spatial(nmodel, "Bar")
plot_spatial(nmodel, "Foo", versus="Bar")  # ratio of Meuse to Rhine